<a href="https://colab.research.google.com/github/SilvanusTurkson/Face-Reg/blob/main/New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install deepface
!pip install deepface streamlit
!streamlit run app.py
!pip install streamlit deepface pillow

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [19]:
import os
import streamlit as st
from deepface import DeepFace
from PIL import Image

# Set page title
st.title("Face Recognition System")

# Function to load and store embeddings of known faces
def load_known_faces(known_faces_dir):
    known_face_embeddings = {}
    for person_name in os.listdir(known_faces_dir):
        person_dir = os.path.join(known_faces_dir, person_name)
        if os.path.isdir(person_dir):
            for img_file in os.listdir(person_dir):
                img_path = os.path.join(person_dir, img_file)
                try:
                    embedding = DeepFace.represent(img_path, model_name="Facenet")[0]["embedding"]
                    known_face_embeddings[person_name] = embedding
                except Exception as e:
                    st.warning(f"Could not process {img_path}: {e}")
    return known_face_embeddings

# Function to recognize an unknown face
def recognize_face(unknown_img_path, known_face_embeddings, threshold=0.6):
    try:
        unknown_embedding = DeepFace.represent(unknown_img_path, model_name="Facenet")[0]["embedding"]
        best_match = None
        best_score = 0

        for name, known_embedding in known_face_embeddings.items():
            similarity = DeepFace.verify(unknown_embedding, known_embedding, model_name="Facenet", distance_metric="cosine")["distance"]
            similarity_score = 1 - similarity  # Convert distance to similarity (0-1)

            if similarity_score > best_score and similarity_score >= threshold:
                best_score = similarity_score
                best_match = name

        if best_match:
            return best_match, best_score
        else:
            return "Unknown", 0
    except Exception as e:
        st.error(f"Error recognizing face: {e}")
        return "Error", 0

# Main Streamlit App
def main():
    # Load known faces (precompute embeddings)
    KNOWN_FACES_DIR = "dataset/known_faces"  # Path to known faces
    known_face_embeddings = load_known_faces(KNOWN_FACES_DIR)

    # Upload or process unknown faces
    st.sidebar.header("Options")
    option = st.sidebar.radio("Choose input:", ("Upload Image", "Process Unknown Faces Folder"))

    if option == "Upload Image":
        uploaded_file = st.file_uploader("Upload a face to recognize", type=["jpg", "png", "jpeg"])
        if uploaded_file:
            img = Image.open(uploaded_file)
            st.image(img, caption="Uploaded Image", width=300)

            # Save temporarily and recognize
            temp_path = "temp.jpg"
            img.save(temp_path)
            name, score = recognize_face(temp_path, known_face_embeddings)

            if name != "Unknown":
                st.success(f"✅ **Match Found:** {name} (Confidence: {score:.2f})")
            else:
                st.warning("❌ No match found.")

    elif option == "Process Unknown Faces Folder":
        UNKNOWN_FACES_DIR = "dataset/unknown_faces"  # Path to unknown faces
        if os.path.exists(UNKNOWN_FACES_DIR):
            st.write("Processing images from 'unknown_faces' folder...")
            for img_file in os.listdir(UNKNOWN_FACES_DIR):
                img_path = os.path.join(UNKNOWN_FACES_DIR, img_file)
                img = Image.open(img_path)
                st.image(img, caption=f"Testing: {img_file}", width=200)

                name, score = recognize_face(img_path, known_face_embeddings)
                if name != "Unknown":
                    st.success(f"✅ **Match Found:** {name} (Confidence: {score:.2f})")
                else:
                    st.warning("❌ No match found.")
                st.write("---")
        else:
            st.error("Folder 'unknown_faces' not found!")

if __name__ == "__main__":
    main()

2025-03-24 18:16:40.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.229 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 18:16:40.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar